In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [1]:
#RUN NEXT 2 CELLS TO OBTAIN IMGS FOR TRAINING AND TEST DATA

In [5]:
backhand_path=r'E:\opencv\tennis2\train\backhand'
backhand_vids=os.listdir(backhand_path)

for vid in backhand_vids:

    vid_name=vid.split('.')[0]
    #print(vid_name)
    cap = cv2.VideoCapture(os.path.join(backhand_path,vid))
    count=0
    while(True):

        ret, frame = cap.read()
        if ret==False:
            break
        file_name_path = backhand_path + '\\' +str(count)+'-'+ str(vid_name) + '.jpg'
        frame_gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(file_name_path, frame_gray)  
        count=count+1

In [6]:
forehand_path=r'E:\opencv\tennis2\train\forehand'
forehand_vids=os.listdir(forehand_path)

for vid in forehand_vids:

    vid_name=vid.split('.')[0]
    #print(vid_name)
    cap = cv2.VideoCapture(os.path.join(forehand_path,vid))
    count=0
    while(True):

        ret, frame = cap.read()
        if ret==False:
            break
        file_name_path = forehand_path + '\\' +str(count)+'-'+ str(vid_name) + '.jpg'
        frame_gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(file_name_path, frame_gray)  
        count=count+1

In [2]:
train_datagen=ImageDataGenerator(rescale=1.0/255.0,horizontal_flip=True)
training_set=train_datagen.flow_from_directory(r'E:\opencv\tennis2\train',target_size=(64,64),batch_size=32,class_mode='binary',color_mode='grayscale')

Found 63065 images belonging to 2 classes.


In [3]:
test_datagen=ImageDataGenerator(rescale=1.0/255.0)
test_set=test_datagen.flow_from_directory(r'E:\opencv\tennis2\test',target_size=(64,64),batch_size=32,class_mode='binary',color_mode='grayscale')

Found 31465 images belonging to 2 classes.


In [4]:
cnn=tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',input_shape=(64,64,1)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dropout(0.5))

cnn.add(tf.keras.layers.Dense(units=256,activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [5]:
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [6]:
#checkpoint=ModelCheckpoint('tennis-{epoch:02d}.model',monitor='val_accuracy',verbose=0,save_best_only=True,mode='auto')

cnn.fit(x=training_set,validation_data=test_set,epochs=10)

Epoch 1/10
1971/1971 [==============================] - 2128s 1s/step - loss: 0.4406 - accuracy: 0.7638 - val_loss: 0.2298 - val_accuracy: 0.9082
Epoch 2/10
1971/1971 [==============================] - 1656s 840ms/step - loss: 0.1498 - accuracy: 0.9404 - val_loss: 0.1429 - val_accuracy: 0.9439
Epoch 3/10
1971/1971 [==============================] - 1604s 814ms/step - loss: 0.0871 - accuracy: 0.9668 - val_loss: 0.1389 - val_accuracy: 0.9539
Epoch 4/10
1971/1971 [==============================] - 1644s 834ms/step - loss: 0.0630 - accuracy: 0.9771 - val_loss: 0.1383 - val_accuracy: 0.9544
Epoch 5/10
1971/1971 [==============================] - 1648s 836ms/step - loss: 0.0468 - accuracy: 0.9831 - val_loss: 0.1367 - val_accuracy: 0.9539
Epoch 6/10
1971/1971 [==============================] - 1608s 816ms/step - loss: 0.0385 - accuracy: 0.9867 - val_loss: 0.1307 - val_accuracy: 0.9609
Epoch 7/10
1971/1971 [==============================] - 1611s 817ms/step - loss: 0.0347 - accuracy: 0.9877 - 

In [9]:
cnn.save("tennis-grayscale-softmax-model.h5")

In [4]:
from keras.models import load_model
model = load_model('tennis-grayscale-softmax-model.h5')

In [7]:
training_set.class_indices

{'backhand': 0, 'forehand': 1}

In [8]:
import numpy as np
from keras.preprocessing import image

path="E:\\opencv\\tennis2\\validation\\"
img_names=os.listdir(path)
for img in img_names:
    test_image=image.load_img(os.path.join(path,img),target_size=(64,64))
    test_img=cv2.cvtColor(np.float32(test_image), cv2.COLOR_BGR2GRAY)
    test_img=image.img_to_array(test_img)
    test_img=np.expand_dims(test_img,axis=0)
    result=cnn.predict(test_img)
    if result[0][0]==0:
        x='Backhand'
    else:
        x='Forehand'
    print(img.split('.')[0]+'-'+img.split('.')[1],x)

backhand-1 Forehand
backhand-10 Forehand
backhand-2 Forehand
backhand-3 Forehand
backhand-4 Forehand
backhand-5 Forehand
backhand-6 Forehand
backhand-7 Forehand
backhand-8 Forehand
backhand-9 Forehand
forehand-1 Forehand
forehand-10 Forehand
forehand-2 Forehand
forehand-3 Forehand
forehand-4 Forehand
forehand-5 Forehand
forehand-6 Forehand
forehand-7 Forehand
forehand-8 Forehand
forehand-9 Forehand


In [4]:
import numpy as np
from keras.preprocessing import image

path="E:\\opencv\\tennis2\\validation\\"
img_names=os.listdir(path)
for img in img_names:
    test_image=image.load_img(os.path.join(path,img),target_size=(64,64))
    #test_img=cv2.cvtColor(np.float32(test_image), cv2.COLOR_BGR2GRAY)
    test_img=image.img_to_array(test_image)
    test_img=np.expand_dims(test_img,axis=0)
    result=model.predict(test_img)
    if result[0][0]==0:
        x='Backhand'
    else:
        x='Forehand'
    print(img.split('.')[0]+'-'+img.split('.')[1],x)

backhand-1 Backhand
backhand-10 Backhand
backhand-2 Forehand
backhand-3 Forehand
backhand-4 Forehand
backhand-5 Backhand
backhand-6 Backhand
backhand-7 Backhand
backhand-8 Forehand
backhand-9 Forehand
forehand-1 Forehand
forehand-10 Backhand
forehand-2 Forehand
forehand-3 Forehand
forehand-4 Forehand
forehand-5 Backhand
forehand-6 Backhand
forehand-7 Forehand
forehand-8 Backhand
forehand-9 Forehand
